# Baseline

In [41]:
import pandas as pd
import numpy as np

In [42]:
def ndcg_func(x,y):
    return (1/np.log(x+2))/(1/np.log(1+y))

In [43]:
df = pd.read_csv("final.csv")
df['zpid'] = df['zpid'].apply(lambda x: x.lstrip(" ") )

In [44]:
df.columns

Index(['Unnamed: 0', 'tracing_id', 'user_id_hash', 'timestamp_session',
       'impressions', 'user_session_id', 'Dates', 'Time', 'order', 'zpid',
       'submit', 'fav', 'click'],
      dtype='object')

In [45]:
df_copy = df
df_copy = df_copy[df_copy["click"]==1]
df_copy = df_copy[df_copy["order"]!= "None"]
df_copy['order'] = df_copy['order'].apply(lambda x: float(x))
df_copy["click_position"] = df_copy.groupby(["tracing_id"])["Time"].rank(method='first')

nDCG

In [46]:
df_ndcg_1 = df_copy.loc[df_copy['click_position'] <= 1]
df_ndcg_2 = df_copy.loc[df_copy['click_position'] <= 2]
df_ndcg_5 = df_copy.loc[df_copy['click_position'] <= 5]
df_ndcg_10 = df_copy.loc[df_copy['click_position'] <= 10]
df_ndcg_20 = df_copy.loc[df_copy['click_position'] <= 20]

In [ ]:
df_ndcg_1.loc[:,"ndcg"] = df_ndcg_1.apply(lambda x: ndcg_func(x.order, x.click_position), axis=1)
df_ndcg_2.loc[:,"ndcg"] = df_ndcg_2.apply(lambda x: ndcg_func(x.order, x.click_position), axis=1)
df_ndcg_5.loc[:,"ndcg"] = df_ndcg_5.apply(lambda x: ndcg_func(x.order, x.click_position), axis=1)
df_ndcg_10.loc[:,"ndcg"] = df_ndcg_10.apply(lambda x: ndcg_func(x.order, x.click_position), axis=1)
df_ndcg_20.loc[:,"ndcg"] = df_ndcg_20.apply(lambda x: ndcg_func(x.order, x.click_position), axis=1)

In [48]:
df_ndcg_1 = df_ndcg_1.groupby("user_session_id")['ndcg'].mean()
df_ndcg_2 = df_ndcg_2.groupby("user_session_id")['ndcg'].mean()
df_ndcg_5 = df_ndcg_5.groupby("user_session_id")['ndcg'].mean()
df_ndcg_10 = df_ndcg_10.groupby("user_session_id")['ndcg'].mean()
df_ndcg_20 = df_ndcg_20.groupby("user_session_id")['ndcg'].mean()

In [49]:
print(df_ndcg_20.mean())
print(df_ndcg_10.mean())
print(df_ndcg_5.mean())
print(df_ndcg_2.mean())
print(df_ndcg_1.mean())

0.5604123683414143
0.5604112440031735
0.5609040850396567
0.5679968247040262
0.5836045662133186


mAP@k

In [50]:
df_precision, df_mmr = df_copy.copy(), df_copy.copy()

In [51]:
df_precision["precision@k"] = df_precision.apply(lambda x: x.click_position/(x.order + 1), axis=1)
df_precision = df_precision.groupby("tracing_id")['precision@k'].mean()
print(df_precision.mean())

0.39452420330835103


MRR

In [52]:
df_mmr = df_mmr.groupby("tracing_id")["order"].min().to_frame().reset_index()
df_mmr["rank"] = df_mmr["order"].apply(lambda x: 1/(x+1))
print(df_mmr["rank"].mean())

0.46536254667501953
